# Tabula Rasa

---

---

### Overview

This notebook trains a mixed monotonic model, with sub-models to generate arbitrary quantile predictions and estimate epistemic uncertainty, using `TabulaRasaRegressor()`.

It's designed to work with Pandas DataFrame's and takes advantage of class types and feature names to cut down on code.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabularasa.TabulaRasa import TabulaRasaRegressor

---

### Load example data

If you haven't already, please generate the example dataset using the [example_data](example_data.ipynb) notebook.

In [2]:
df = pd.read_pickle('./data/simple_train.pkl')

In [3]:
df.dtypes

y     float32
x1    float32
x2    float32
x3      int64
x4    float32
dtype: object

Let's convert `x3` to a `category` data type to generate embeddings for it (`TabulaRasaRegressor()` automatically handles this for all columns with `object` or `category` data types).

In [4]:
df['x3'] = df['x3'].astype('category')

---

### Initialize model

When initializing the model, we typically pass 3 arguments:
- `df`: A `pandas.DataFrame` containing the training data, or a sample of it.  No training happens on initialization, just categorizing features, setting up categorical feature mappings, and scalers for numeric features.  Therefore, if it is a sample of the full dataset, it should well represent your full dataset (in terms of having unique values for each categorical feature, and distributions for continuous features).
- `targets`: A `list` of column names to use as regressand(s) which are in `df`.  All other 'number', 'category', or 'object' columns in `df` are assumed to be features and will be included in the models. 
- `monotonic_constraints`: A `dict` where keys are features (column names on `df`) to take on monotonic relationships with the `targets` and values are 1 or -1 to signify the direction of that relationship: increasing or decreasing (respectively).

In [5]:
trr = TabulaRasaRegressor(df,
                          targets=['y'],
                          monotonic_constraints={'x1': 1, 'x2': 1})

In [6]:
trr.fit(df)

*** Training core model ***


AssertionError: you must pass in 2 values for your categories input